In [1]:
import numpy as np
import pandas as pd
import pickle
from glob import glob
import gzip
from tqdm.notebook import tqdm
from haversine import haversine

# Preprocess Data (Fwy 405 N)
Creates a simplified data set that will be good for model testing.  
Selects only stations on Fwy 405 N (the main freeway in LA)  
Creates an adjacency matrix for these stations only and a concatenated csv with 3 months of speed data   
Files are stored in `/data/processed/fwy-405-n/`

### Setup

In [2]:
# read station metadata
meta = pd.read_csv('../data/raw_pems/d07_text_meta_2021_10_20.txt', sep='\t')
meta = meta[['ID', 'Fwy', 'Dir', 'District', 'County', 'City', 'State_PM', 'Abs_PM',
             'Latitude', 'Longitude', 'Length', 'Type', 'Lanes', 'Name']]

meta.head()

,ID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name
0,715898,5,S,7,37,40032.0,.71,117.280,33.880183,-118.021787,0.43,ML,3,PHOEBE
1,715900,5,S,7,37,40032.0,1.06,117.630,33.882892,-118.026822,NaN,OR,1,VALLEY VIEW
2,715901,5,N,7,37,40032.0,1.11,117.743,33.883400,-118.027451,NaN,OR,1,VALLEY VIEW
3,715903,5,N,7,37,69154.0,1.56,118.193,33.886992,-118.034125,NaN,OR,1,ALONDRA
4,715904,5,S,7,37,69154.0,2.27,118.840,33.892489,-118.044573,NaN,OR,1,CARMENITA


In [3]:
# read station data helper function
data_dict = pd.read_csv('../data/raw_pems/station_dict.csv', names=['Feature', 'Description', 'Unit']).iloc[1:].reset_index(drop=True)
colnames = data_dict['Feature'].values.tolist()
colnames_subset = ['Timestamp', 'Station', 'Station Length', 'Samples', 'Total Flow', 'Avg Occupancy', 'Avg Speed', 
                'Lane N Samples', 'Lane N Flow', 'Lane N Avg Occ', 'Lane N Avg Speed', 'Lane N Observed']

FILES = sorted(glob('../data/raw_pems/*.gz'))

def read_station_data(file_no):

    with gzip.open(FILES[file_no]) as f:
        df = pd.read_csv(f, index_col=False, names=colnames)[colnames_subset]

    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    return df

### Filter Data
Only use sensors along Fwy 405 N route  
Build adjacency matrix using absolute postmile

In [4]:
# select only Fwy 405 N mainlines
meta = meta[(meta['Fwy'] == 405) & (meta['Type'] == 'ML') & (meta['Dir'] == 'N')]
meta = meta.sort_values('Abs_PM')
meta = meta.reset_index(drop=True)
meta.head()

,ID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name
0,771826,405,N,7,37,43000.0,.11,24.058,33.787054,-118.094505,0.303,ML,4,N. OF 605
1,717696,405,N,7,37,43000.0,.6,24.548,33.791829,-118.100867,0.495,ML,4,STUDEBAKER
2,718219,405,N,7,37,43000.0,1.1,25.048,33.796718,-118.107431,0.565,ML,4,PALO VERDE
3,717701,405,N,7,37,43000.0,1.73,25.678,33.802431,-118.115997,0.675,ML,4,WOODRUFF
4,717703,405,N,7,37,43000.0,2.45,26.398,33.802865,-118.128314,0.505,ML,4,BELLFLOWER 2


In [5]:
from folium import Map
import folium.plugins as plugins
import folium

In [6]:
dir_mapper = {'S': 'south', 'N': 'north', 'E': 'east', 'W': 'west'}
m = Map(location=(34.0522,-118.2437), tiles='https://tile.jawg.io/jawg-dark/{z}/{x}/{y}{r}.png?access-token=yxQukjQJyY3mRrF6htcGR22i1QJ6BP6wslSe2Cmq2k4aT8S0wbDtYMEaPhc8s240', attr='<a href="http://jawg.io" title="Tiles Courtesy of Jawg Maps" target="_blank">&copy; <b>Jawg</b>Maps</a> &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors', control_scale=True, max_bounds=True, prefer_canvas=True, zoom_start=10)
for ind, row in meta.iterrows():
    icon = folium.features.CustomIcon(f"../data/icons/{(dir_mapper[row['Dir']])}.png", icon_size=(15, 15))
    style = 'font-family: Impact, Haettenschweiler, "Franklin Gothic Bold", Charcoal, "Helvetica Inserat", "Bitstream Vera Sans Bold", "Arial Black", "sans serif";'
    html = f'''<body style="{style}"><div align="center", style="background-color: #BAD6FF; font-family: Arial">''' + row.to_frame().transpose()[['ID', 'Fwy', 'Length', 'Type', 'Lanes']].transpose().to_html(justify="center", header=False, index=True, index_names=False, col_space=300, classes="table-condensed table-responsive table-success") + '</div></body>' 
    popup = folium.Popup(html, max_width=300)                
    folium.Marker(row[['Latitude', 'Longitude']].values.tolist(), icon=icon, popup=popup).add_to(m)
#m.save('../plots/fwy_405_n_ds_stations.html')

In [13]:
# build adjacency matrix
# we build many different types here
# choose which one is saved at the end of the notebook

stations = np.array(meta['ID'])

# build station-ind mapper
station_ind_mapper = {station: i for i, station in enumerate(stations)}

# haversine distance connecting all stations
mat_haversine = np.array( [[haversine((float(row1['Latitude']), float(row1['Longitude'])), \
                                      (float(row2['Latitude']), float(row2['Longitude']))) \
                           for i1, row1 in meta.iterrows()] for i2, row2 in meta.iterrows()] )

# postmile distance connecting all stations
mat_all = np.array( [[abs(y-x) for y in meta['Abs_PM']] for x in meta['Abs_PM']] )

# distance connecting only upstream stations; uses manhattan as distance metric
mat_upstream = np.array( [[y-x if y-x>=0 else np.inf for y in meta['Abs_PM']] for x in meta['Abs_PM']] )

# distance connecting only adjacent stations; uses manhattan as distance metric
mat_adjacent = np.array( [[abs(x-y) if abs(i-j)<=1 else np.inf for (i,y) in enumerate(meta['Abs_PM'])] \
                                                         for (j,x) in enumerate(meta['Abs_PM'])] )

print(mat_haversine)

[[ 0.          0.79219735  1.60672239 ... 62.73669169 64.93084452
  65.40730117]
 [ 0.79219735  0.          0.81452781 ... 61.96946942 64.16940767
  64.64736247]
 [ 1.60672239  0.81452781  0.         ... 61.18205913 63.38815041
  63.8676968 ]
 ...
 [62.73669169 61.96946942 61.18205913 ...  0.          2.81791876
   3.46434814]
 [64.93084452 64.16940767 63.38815041 ...  2.81791876  0.
   0.64828939]
 [65.40730117 64.64736247 63.8676968  ...  3.46434814  0.64828939
   0.        ]]


### Extract Station Data
Get a single dataframe with time and speed for the selected sensors

In [17]:
# function to extract time x speed dataframe for selected stations
def get_station_data(start_day, end_day):
    data = []
    for i in tqdm( range(start_day, end_day) ):
        df = read_station_data(i)
        df = df[df['Station'].isin(meta['ID'])]
        df = df[['Timestamp', 'Station', 'Avg Speed']]
        df.columns = ['Time', 'Station', 'Speed']
        data.append(df)
    
    return pd.concat(data).reset_index(drop=True)

# select all of 2021
df = get_station_data(0, len(FILES))
df

,Time,Station,Speed
0,2021-01-01 00:00:00,716632,71.4
1,2021-01-01 00:00:00,716659,68.6
2,2021-01-01 00:00:00,716663,68.5
3,2021-01-01 00:00:00,716670,67.9
4,2021-01-01 00:00:00,716689,65.3
...,...,...,...
11044371,2021-12-31 23:55:00,775261,71.0
11044372,2021-12-31 23:55:00,775936,64.6
11044373,2021-12-31 23:55:00,776739,64.9
11044374,2021-12-31 23:55:00,776844,69.4


In [18]:
# create pivoted df with column for each station
pivoted_df = df.pivot_table(values='Speed', index='Time', columns='Station')
pivoted_df.index.name = None
pivoted_df.columns.name = None
pivoted_df.index = pd.to_datetime(pivoted_df.index)
pivoted_df = pivoted_df[stations]
pivoted_df

,771826,717696,718219,717701,717703,718227,717706,717709,716632,771845,...,767351,717819,717823,767367,717825,717827,771808,771767,772011,772024
2021-01-01 00:00:00,69.2,68.6,69.9,70.8,68.7,70.5,72.6,72.2,71.4,69.2,...,64.9,68.7,68.1,67.7,68.1,67.8,64.9,71.8,69.3,66.8
2021-01-01 00:05:00,70.4,68.2,67.9,69.4,66.9,70.6,72.2,73.7,71.5,69.4,...,64.9,69.6,68.7,68.4,69.4,68.4,63.8,68.8,69.4,65.0
2021-01-01 00:10:00,70.1,69.3,69.4,70.5,67.4,70.7,70.5,73.3,71.6,70.0,...,65.2,69.2,68.2,67.8,68.7,67.5,65.0,67.5,69.8,67.1
2021-01-01 00:15:00,69.9,68.7,69.1,69.9,70.5,70.4,71.9,75.4,71.6,69.8,...,64.7,69.4,68.4,68.3,69.9,68.8,67.0,72.8,69.0,66.3
2021-01-01 00:20:00,70.3,69.5,68.9,70.6,59.8,71.0,72.5,74.5,71.9,70.8,...,65.0,70.3,68.9,68.7,69.9,68.1,69.8,72.6,69.1,65.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 23:35:00,70.6,69.5,69.6,71.0,70.2,72.6,68.2,64.7,71.5,70.7,...,65.4,70.7,66.9,64.2,66.8,68.6,68.6,63.2,69.1,69.4
2021-12-31 23:40:00,71.1,69.5,69.8,72.2,73.6,73.1,66.5,69.0,72.0,70.9,...,65.1,70.1,67.1,64.1,66.9,68.7,68.7,63.4,69.7,68.6
2021-12-31 23:45:00,70.8,70.0,70.2,64.0,72.9,73.1,67.0,75.0,72.2,71.0,...,65.4,71.0,67.2,64.3,68.1,68.8,68.8,63.3,69.9,68.3
2021-12-31 23:50:00,70.8,70.0,70.2,69.6,65.6,73.5,66.4,75.6,72.5,70.2,...,65.7,71.1,67.4,64.4,67.6,68.9,68.9,63.2,70.0,68.0


In [22]:
# ensure there are no stations in adj mat that aren't in pivoted df
station_ind_mapper_subs = {station: station_ind_mapper[station] for station in station_ind_mapper if station in pivoted_df.columns}
ind_to_delete = [i for i in range(mat_haversine.shape[0]) if i not in station_ind_mapper_subs.values()]
len(ind_to_delete)

0

### Ensure Speed Data Has a Consistent Frequency

In [23]:
pivoted_df.index.value_counts()

2021-09-19 09:45:00    1
2021-03-20 17:35:00    1
2021-10-29 01:55:00    1
2021-02-28 13:50:00    1
2021-10-31 07:40:00    1
                      ..
2021-01-19 14:40:00    1
2021-01-19 01:25:00    1
2021-08-25 01:40:00    1
2021-11-05 13:20:00    1
2021-11-12 18:30:00    1
Length: 105077, dtype: int64

There are no duplicate dates. However, we can see that some 5-min intervals are missing in the year.

In [24]:
set(pd.date_range('2021-01-01', '2022-01-01', freq='5T')) - set(pivoted_df.index)

{Timestamp('2021-01-05 20:35:00', freq='5T'),
 Timestamp('2021-02-16 07:25:00', freq='5T'),
 Timestamp('2021-03-14 02:00:00', freq='5T'),
 Timestamp('2021-03-14 02:05:00', freq='5T'),
 Timestamp('2021-03-14 02:10:00', freq='5T'),
 Timestamp('2021-03-14 02:15:00', freq='5T'),
 Timestamp('2021-03-14 02:20:00', freq='5T'),
 Timestamp('2021-03-14 02:25:00', freq='5T'),
 Timestamp('2021-03-14 02:30:00', freq='5T'),
 Timestamp('2021-03-14 02:35:00', freq='5T'),
 Timestamp('2021-03-14 02:40:00', freq='5T'),
 Timestamp('2021-03-14 02:45:00', freq='5T'),
 Timestamp('2021-03-14 02:50:00', freq='5T'),
 Timestamp('2021-03-14 02:55:00', freq='5T'),
 Timestamp('2021-04-11 17:25:00', freq='5T'),
 Timestamp('2021-08-07 23:00:00', freq='5T'),
 Timestamp('2021-08-26 03:45:00', freq='5T'),
 Timestamp('2021-08-30 04:20:00', freq='5T'),
 Timestamp('2021-10-18 05:05:00', freq='5T'),
 Timestamp('2021-12-19 22:00:00', freq='5T'),
 Timestamp('2021-12-19 22:05:00', freq='5T'),
 Timestamp('2021-12-19 22:10:00', 

In [25]:
pivoted_df_5t_freq = pivoted_df.reindex(pd.date_range('2021-01-01', '2022-01-01', freq='5T'))

In [26]:
# identify missing values
print('missing values: ', pivoted_df_5t_freq.isna().sum().sum())

pivoted_df_5t_freq[ [x.isna().any() for x in pivoted_df_5t_freq.iloc] ]

missing values:  5089


,771826,717696,718219,717701,717703,718227,717706,717709,716632,771845,...,767351,717819,717823,767367,717825,717827,771808,771767,772011,772024
2021-01-05 20:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-02-16 07:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-02-18 18:40:00,NaN,64.4,64.4,64.5,NaN,65.0,64.0,NaN,65.2,65.1,...,NaN,NaN,NaN,35.2,NaN,NaN,NaN,57.9,NaN,64.7
2021-03-09 15:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,25.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-14 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-14 02:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-14 02:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-14 02:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-14 02:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-14 02:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
pivoted_df_5t_freq.apply(lambda x: (x.isna().sum() / pivoted_df_5t_freq.shape[0]) * 100).sort_values()

717740    0.042808
717696    0.043759
717701    0.043759
718227    0.043759
717752    0.043759
            ...   
716663    0.049467
771877    0.049467
777065    0.049467
761492    0.049467
759427    0.049467
Length: 105, dtype: float64

In [28]:
# fill missing values with linear interpolation
pivoted_df_5t_freq = pivoted_df_5t_freq.interpolate(method='linear', limit_direction='forward', axis=0)

### Save Data

location: `data/processed/fwy_405_n_ds/`

In [29]:
# write station metadata file
meta.to_csv('../data/processed/fwy_405_n_ds/meta.csv', index=False)

# save adj_mat 
with open('../data/processed/fwy_405_n_ds/adj_mat.dat', 'wb')  as f:
    pickle.dump(mat_haversine, f)

# save mapper for adj_mat index to station ID
with open('../data/processed/fwy_405_n_ds/adj_mat_ind_station_mapper.dat', 'wb') as f:
    pickle.dump({station_ind_mapper[station]: station for station in station_ind_mapper}, f)

# save full speed df concatenated for all days in 2021
with open('../data/processed/fwy_405_n_ds/speeds.dat', 'wb')  as f:
    pickle.dump(pivoted_df_5t_freq, f)